In [ ]:
# プロジェクト直下で notebook を開いている想定:
# project/
# ├─ relicot/
# │   ├─ CoT/
# │   └─ note/test.ipynb  ← これ
import sys, pathlib
root = pathlib.Path(__file__).resolve().parents[1] if "__file__" in globals() else pathlib.Path.cwd().parents[0]
sys.path.insert(0, str(root))  # プロジェクト直下を import パスに追加
print("PYTHONPATH added:", root)

PYTHONPATH added: /content/drive/MyDrive/Colab Notebooks


In [ ]:
# === 0) 既存のモデル準備（あなたのセルをそのまま使ってOK） ===
# !pip install transformers accelerate torch --quiet

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

def hf_generate(prompt: str, max_new_tokens=256, temperature=0.7, top_p=0.95, stop=None):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if text.startswith(prompt):
        text = text[len(prompt):].lstrip()
    if stop:
        for s in ([stop] if isinstance(stop, str) else stop):
            idx = text.find(s)
            if idx >= 0:
                text = text[:idx]
                break
    return text


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 0) 既存のモデル読込（あなたのセルそのままでOK）
# tokenizer, model, hf_generate(...)

# 1) CoTGenerator
from relicot.CoT.cot_module import CoTGenerator
gen = CoTGenerator(generate_fn=hf_generate, max_input_tokens=4096)

# 2) UQ: scorer とパイプライン
from UQ.uq_module import HFCausalLogProbScorer, run_cotuq
scorer = HFCausalLogProbScorer(model=model, tokenizer=tokenizer)

# 3) 実行
q = "10個のリンゴを3人でできるだけ均等に分けると、一人あたりいくつ？"
out = run_cotuq(
    gen=gen,
    question=q,
    tokenizer=tokenizer,
    scorer=scorer,
    ap_use=True,
    ap_agg="mean",
    ap_tau=7,                # KEYKeywords 的フィルタを有効にするなら
    se_modes=["ALLSteps","KEYStep","KEYKeywords"],
    se_tau=7,
)

print("Answer:", out.answer)
print("Steps:", out.steps)
print("\n[Stage1] Keywords+Scores by Step:")
for i in sorted(out.kw_by_step.keys()):
    print(f"Step {i}:", out.kw_by_step[i])

print("\n[Stage2] AP (keyword-weighted):", out.ap_confidence)
print("[Stage2] SE (P(True)):", out.se_confidences)


Answer: 一人あたり3個のリンゴとなりますが、1個のリンゴは均等に分配できません。実際の分配では2人には3個ずつ、残り1人が2個と1個になります。 Answer: 一人あたり3個 (実際の分配では2人3個、1人2個と1個)。 ただし、問題文
Steps: ['10個のリンゴを3人でできるだけ均等に分ける場合を考える。', '均等分配を実現するために、リンゴをできるだけ等しい数で割る。', '商が得られた結果から、一人あたりのリンゴ数を決定する。', '余りがある場合、均等に分配しきれないことを確認する。']

[Stage1] Keywords+Scores by Step:
Step 1: [('リンゴ (/score/)', 1), ('3人 (/score/)', 1), ('均等 (/score/)', 1)]
Step 2: [('均等 (/score/)', 1), ('分配 (/score/)', 1)]
Step 3: [('商 (/score/)', 1), ('一人 (/score/)', 1), ('果物数 (/score/)', 1)]
Step 4: [('余り (/score/)', 1), ('均等 (/score/)', 1)]

[Stage2] AP (keyword-weighted): 1.0536940590647677e-05
[Stage2] SE (P(True)): {'ALLSteps': 0.6800292550948656, 'KEYStep': 0.5926665999540697, 'KEYKeywords': 0.6215409097858049}
